In [ ]:
import os
import time
import requests
import psycopg2  # Assuming you're using psycopg2 for PostgreSQL connection

# Database connection setup
conn = psycopg2.connect(
    dbname=os.getenv("DB_NAME"),
    user=os.getenv("DB_USER"),
    password=os.getenv("DB_PASSWORD"),
    host=os.getenv("DB_HOST")
)
cursor = conn.cursor()

# Set up DataHub API variables
datahub_url = os.getenv("DATAHUB_SERVER_URL", "http://35.177.132.152:8080") + "/api/v3/entity/dataset"
headers = {
    "Authorization": f"Bearer {os.getenv('DATAHUB_TOKEN')}",
    "Content-Type": "application/json"
}

# Query datasets
cursor.execute("SELECT urn FROM your_dataset_table")  # Replace with your actual table and column
rows = cursor.fetchall()


In [ ]:

# Process each dataset and send the result to DataHub
for row in rows:
    urn = row[0]
    completeness_status = 'FAILURE'  # No owner found, hence 'FAILURE'
    timestamp = int(time.time() * 1000)  # Current time in milliseconds

    # Create payload
    payload = {
        "value": {
            "failing": [
                {
                    "test": "urn:li:test:metadata_completeness_check",
                    "type": completeness_status,
                    "testDefinitionMd5": "someUniqueMd5Hash",
                    "lastComputed": {
                        "time": timestamp,
                        "actor": "urn:li:corpuser:datahub"
                    }
                }
            ],
            "passing": []
        },
        "systemMetadata": {}
    }

    # First try to create
    response = requests.post(f"{datahub_url}/{urn}/testresults", headers=headers, json=payload)

    # If the aspect already exists, try to update
    if response.status_code == 409:  # Conflict - aspect exists
        print(f"Aspect already exists for {urn}. Trying to update...")
        response = requests.post(f"{datahub_url}/{urn}/testresults?createIfNotExists=false", headers=headers, json=payload)

    # Check the response status
    if response.status_code in [200, 201]:
        print(f"Successfully posted test result for {urn}")
    else:
        print(f"Failed to post test result for {urn}: {response.status_code} - {response.content.decode() if response.content else 'No content returned'}")

# Clean up resources
cursor.close()
conn.close()
